In [1]:
!pip install -q gpt-2-simple
!pip install fpdf
!pip install diffusers -q
!pip install accelerate - q
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import random

ERROR: Invalid requirement: '-'


In [2]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 3.47Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 3.20Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 3.60Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:12, 41.3Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 2.59Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.51Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.67Mit/s]


In [3]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_name = 'All Speaches.txt'

In [5]:
gpt2.copy_file_from_gdrive(file_name)

In [6]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='latest',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=10,
              )

Loading checkpoint checkpoint/run1/model-101
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


dataset has 82295 tokens
Training...
[110 | 1084.44] loss=1.61 avg=1.61
Saving checkpoint/run1/model-110
[120 | 2264.27] loss=1.65 avg=1.63
Saving checkpoint/run1/model-120


Instructions for updating:
Use standard file APIs to delete files with this prefix.


[130 | 3426.99] loss=1.45 avg=1.57
Saving checkpoint/run1/model-130
interrupted
Saving checkpoint/run1/model-131


In [20]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [6]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [7]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-90


In [21]:
Word_line = int(input("How many words per line? "))
Prompt = input("Write the first line here (using the same number of words entered above) ")
number_lines = int(input("How many lines? "))
Randomness = input("How crazy do you want the book to be? (mild, moderate, spicy) ")
book_length = int((Word_line*number_lines)*1.25)
Rhyming = input("Rhyming scheme? (y/n)")
if Randomness == "mild":
  temperature = 0.5
elif Randomness == "moderate":
  temperature = 0.75
else:
  temperature = 1

How many words per line? 6
Write the first line here (using the same number of words entered above) I like to eat breakfast pizza
How many lines? 10
How crazy do you want the book to be? (mild, moderate, spicy) moderate
Rhyming scheme? (y/n)y


In [12]:
output_file = 'childrens_book.txt'
book_length = int(book_length)
generated_samples = gpt2.generate(sess,
              prefix = Prompt,
              length=book_length,
              top_k = 40,
              temperature=temperature,
              batch_size=1,
              top_p = 0.95,
              return_as_list = True
              )
print(generated_samples)

generated_book = '\n'.join(generated_samples)

with open(output_file, 'w') as file:
    file.write(generated_book)


["The stormy clouds sent the rats running for the exits. The only thing left of them was the smell of blood.\nDazed, the rats sprinted towards the tent before they could even make it to the tent itself. It was a long day in the city, and the city was littered with the smell of bodies. It wasn't long until the smell finally left the tent and the dogs bolted."]


In [25]:
# split text to create pages

import random

def split_text(input_text, words_per_segment):
    words = input_text.split()
    segments = [words[i:i + words_per_segment] for i in range(0, len(words), words_per_segment)]
    return segments

input_file = 'childrens_book.txt'
output_file_prefix = 'output_segment_'
words_per_segment = random.randint(20,30)

with open(input_file, 'r') as file:
    content = file.read()

text_segments = split_text(content, words_per_segment)


for i, segment in enumerate(text_segments):
    output_file = f'{output_file_prefix}{i + 1}.txt'
    with open(output_file, 'w') as file:
        file.write(' '.join(segment))

    print(f"Segment {i + 1} saved to {output_file}")

seg_count = len(text_segments)
print("Page count = " seg_count)

Segment 1 saved to output_segment_1.txt
Segment 2 saved to output_segment_2.txt
Segment 3 saved to output_segment_3.txt
Segment 4 saved to output_segment_4.txt
Segment 5 saved to output_segment_5.txt
Segment 6 saved to output_segment_6.txt
Segment 7 saved to output_segment_7.txt
Segment 8 saved to output_segment_8.txt
Segment 9 saved to output_segment_9.txt
Segment 10 saved to output_segment_10.txt
10


In [ ]:
from diffusers import DiffusionPipeline
ldm = DiffusionPipeline.from_pretrained(
    "CompVis/ldm-text2im-large-256"
    )

output = ldm(
     [Prompt],
     num_inference_steps=50,
     eta=0.3,
     guidance_scale=6
      )
output["images"][0].save("image.png")

In [17]:
# Code for Generating PDF with split text

class CustomPDF(FPDF):
    def header(self):
        pass

    def footer(self):
        pass

pdf = CustomPDF()

pdf.add_page()
pdf.set_font("Arial", size=16)

input_files = [f'output_segment_{i + 1}.txt' for i in range(seg_count)]

for input_file in input_files:
    pdf.add_page()
    with open(input_file, "r", encoding="utf-8") as file:
        content = file.readlines()

    x_centered = (pdf.w - pdf.get_string_width(content[0].strip())) / 2

    for page_num, page_content in enumerate(content, 1):
        chunks = [page_content[i:i + 50] for i in range(0, len(page_content), 50)]
        image_path = "image.png"
        pdf.image(image_path, x=10, y=10, w=190)
        for chunk in chunks:
            pdf.cell(0, 10, txt=chunk.strip(), ln=True, align='C')  # Use ln=True to add a line break

pdf.output("new2_combined_output.pdf")





ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

<ipython-input-17-d51a16acf4c2>:17: UserWarning: Substituting font arial by core font helvetica
  pdf.set_font("Arial", size=16)


NameError: name 'seg_count' is not defined

In [18]:
# Basic PDF
from fpdf import FPDF

class CustomPDF(FPDF):
    def header(self):
        pass

    def footer(self):
        pass

pdf = CustomPDF()

pdf.add_page()
pdf.set_font("Times", size=12)

text_file_path = "/content/childrens_book.txt"


with open(text_file_path, "r", encoding="utf-8") as file:
    content = file.read()

breaks = content.splitlines()

for line_num, line in enumerate(breaks, 1):
    words = line.split()
    chunks = [words[i:i + Word_line] for i in range(0, len(words), Word_line)]

    for chunk in chunks:
        pdf.cell(0, 10, txt=" ".join(chunk), ln=True)

    if line_num % 4 == 0:
        pdf.ln()
        pdf.ln()

pdf.multi_cell(0, 10, txt=content)

pdf.add_page()
image_path = "image.png"
pdf.image(image_path, x=10, y=10, w=190)

output_pdf_path = "output.pdf"
pdf.output(output_pdf_path)



<ipython-input-18-f7dcbaf83286>:29: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt=" ".join(chunk), ln=True)
<ipython-input-18-f7dcbaf83286>:29: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt=" ".join(chunk), ln=True)
<ipython-input-18-f7dcbaf83286>:35: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(0, 10, txt=content)


In [23]:
# Adding Ryming words for poem

if Rhyming == 'y':
  import requests
  import random

  def get_rhyming_words(word):
      url = requests.get(f'https://api.datamuse.com/words?rel_rhy={word}')
      rhyming_words = [result['word'] for result in url.json()]
      return rhyming_words



  # Ryming with last word of user input

  if len(Prompt) >= Word_line:
      chosen_word = Prompt.split()[Word_line - 1]

  text_file_path = "/content/childrens_book.txt"

  def insert_rhyming_words(text, rhyming_words, interval):
      words = text.split()
      index = (Word_line*2) - 1

      while index < len(words):
          selected_word = random.choice(rhyming_words)
          words.insert(index, selected_word)
          index += interval + 1

      return " ".join(words)

  with open(text_file_path) as file:
      content = file.read()

  rhyming_words = get_rhyming_words(chosen_word)

  updated_content = insert_rhyming_words(content, rhyming_words, interval=Word_line*2)

  with open(text_file_path, "w", encoding="utf-8") as file:
      file.write(updated_content)

  print(chosen_word)
else:
  print("oops")

Rhyming words inserted every 12 words in '/content/childrens_book.txt'.
pizza


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!apt-get install -y git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [27]:
!git clone <https://github.com/henrybatson/Level_up_program.git>


/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `git clone <https://github.com/henrybatson/Level_up_program.git>'
